# 단층, 다층 퍼셉트론 구현하기

단층 퍼셉트론으로 XOR 문제가 풀이가 가능할지 구현한다.

또한, 다층 퍼셉트론으로 XOR 문제가 풀이가 가능할지 구현한다.

퍼셉트론 개념: https://www.notion.so/noversezero/Perceptron-5e5516a2c5ee42d59f2f8f854dcc898d

퍼셉트론 XOR 문제: https://www.notion.so/noversezero/XOR-a68e503051f740c6b7649e40fdc13dd0

다층 퍼셉트론 개념: https://www.notion.so/noversezero/MLP-d57ac1a6d3024fa2a36675b4dd65e609

In [5]:
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

CPU로 할것인지 GPU로 할 것인지 확인하는 구문

In [6]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = torch.FloatTensor([[0], [1], [1], [0]])

XOR 문제를 확인하기 위해 tensor를 만든다.

  X   =  Y
  
[0,0] = [0]

[0,1] = [1]

[1,0] = [1]

[1,1] = [0]

## 단층 퍼셉트론으로 XOR 문제 풀기

In [7]:
linear = nn.Linear(2, 1, bias=True) #input 2개 output 1개의 Linear model
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear, sigmoid) #model의 Active Function은 Sigmoid

In [8]:
# 비용 함수와 옵티마이저 정의
criterion = torch.nn.BCELoss() # Binary Cross Entropy로 정의
optimizer = torch.optim.SGD(model.parameters(), lr=1) # learning rate = 1,확률적 경사하강법

In [12]:
#10,001번의 에포크 수행. 0번 에포크부터 10,000번 에포크까지.
for step in range(10001): 
    optimizer.zero_grad()
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 1000 == 0: # 1000번째 에포크마다 비용 출력
        print(step, cost.item())

0 0.6931471824645996
1000 0.6931471824645996
2000 0.6931471824645996
3000 0.6931471824645996
4000 0.6931471824645996
5000 0.6931471824645996
6000 0.6931471824645996
7000 0.6931471824645996
8000 0.6931471824645996
9000 0.6931471824645996
10000 0.6931471824645996


In [10]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis):  [[0.5]
 [0.5]
 [0.5]
 [0.5]]
모델의 예측값(Predicted):  [[0.]
 [0.]
 [0.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  0.5


## 다층 퍼셉트론으로 XOR 문제 풀기

단층 퍼셉트론과 같지만 model을 만들때 Sequential을 다층으로 해준다.

In [17]:
model = nn.Sequential(
          nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
          nn.Sigmoid(),
          nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
          nn.Sigmoid()
          ).to(device)

In [20]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1

In [21]:
for epoch in range(10001):
    optimizer.zero_grad()
    # forward 연산
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    # 1000의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 1000 == 0:
        print(epoch, cost.item())

0 0.695870041847229
1000 0.6931446194648743
2000 0.6931255459785461
3000 0.6930868029594421
4000 0.6929389834403992
5000 0.6887685060501099
6000 0.001516415155492723
7000 0.0005303305806592107
8000 0.0003135718870908022
9000 0.0002205626224167645
10000 0.0001693219819571823


In [22]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis):  [[1.2716786e-04]
 [9.9983418e-01]
 [9.9984252e-01]
 [2.2669900e-04]]
모델의 예측값(Predicted):  [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0
